
# **COMP9414 Artificial Intelligence**
## Tutorial 3: Constraint Satisfaction

@Author: **Wayne Wobcke**

### Objective

This week we look at algorithms for solving Constraint Satisfaction Problems (CSPs). This will be essential preparation for Assignment 1.

### Before the tutorial

Review the lectures on Constraint Satisfaction Problems.

Read the AIPython documentation on the CSP solvers (the pdf file in the `aipython` directory) and look at the CSP solver code.

Load into VS Code (or wherever you run your Python), the file `cspConsistencyGUI.py`.

Run this code separately to step through the arc consistency algorithm.

Finally, look at the code for the N-Queens problem in `cspExamples.py` and fix the mistake in the definition of `queens(ri,rj)` (see lecture slides).

### 1. Set up AIPython

In this setup, the Jupyter notebook is on my desktop, and the aipython directory is also on my desktop.

To find the aipython files, the aipython directory has to be added to the Python path.

You can do this by changing the shell environment variable PYTHONPATH (permanent option), or temporarily, as done here.

You can add either the full path (using `os.path.abspath`), or as in the code below, the relative path.

In [19]:
import sys
sys.path.append('aipython')
sys.path # check that aipython is now on the path

['c:\\Users\\35562\\.conda\\envs\\com9414\\python313.zip',
 'c:\\Users\\35562\\.conda\\envs\\com9414\\DLLs',
 'c:\\Users\\35562\\.conda\\envs\\com9414\\Lib',
 'c:\\Users\\35562\\.conda\\envs\\com9414',
 '',
 'c:\\Users\\35562\\.conda\\envs\\com9414\\Lib\\site-packages',
 'c:\\Users\\35562\\.conda\\envs\\com9414\\Lib\\site-packages\\win32',
 'c:\\Users\\35562\\.conda\\envs\\com9414\\Lib\\site-packages\\win32\\lib',
 'c:\\Users\\35562\\.conda\\envs\\com9414\\Lib\\site-packages\\Pythonwin',
 'E:\\onedrive\\OneDrive - UNSW\\桌面\\CMP9414\x07ipython',
 'e:\\onedrive\\OneDrive - UNSW\\桌面\\CMP9414\\aipython',
 'E:\\onedrive\\OneDrive - UNSW\\桌面\\CMP9414\x07ipython',
 'E:\\onedrive\\OneDrive - UNSW\\桌面\\CMP9414\\aipython',
 'aipython',
 'aipython',
 'aipython',
 'aipython',
 'aipython',
 'aipython']

### 2. Depth-First Search Constraint Solver

The Depth-First Constraint Solver in AIPython by default uses a random ordering of the variables in the CSP.

We will use the N-Queens problem as a running example in this tutorial. This is defined in `cspExamples.py`.

Run the solver by calling `dfs_solve1` (first solution) or `dfs_solve_all` (all solutions).

In [20]:
from cspExamples import n_queens
from cspDFS import dfs_solve1, dfs_solve_all

# call the Depth-First Search solver
print(dfs_solve1(n_queens(4)))
# print(dfs_solve_all(n_queens(4)))

{R0: 1, R1: 3, R2: 2, R3: 0}


### 3. Depth-First Search Constraint Solver using Forward Checking with MRV Heuristic

The Depth-First Constraint Solver in AIPython by default uses a random ordering of the variables in the CSP.

We redefine the `dfs_solver` methods to implement the MRV (minimum remaining values) heuristic using forward checking.

Because the AIPython code is designed to manipulate domain sets, we also need to redefine `can_evaluate` to handle partial assignments.

In [21]:
num_expanded = 0
display = False

def can_evaluate(c, assignment):
    """ assignment is a variable:value dictionary
        returns True if the constraint can be evaluated given assignment
    """
    return assignment != {} and all(v in assignment.keys() and type(assignment[v]) != list for v in c.scope)

def mrv_dfs_solver(constraints, context, var_order):
    """ generator for all solutions to csp
        context is an assignment of values to some of the variables
        var_order  is  a list of the variables in csp that are not in context
    """
    global num_expanded, display
    if display:
        print("Context", context)
    to_eval = {c for c in constraints if can_evaluate(c, context)}
    if all(c.holds(context) for c in to_eval):
        if var_order == []:
            print("Nodes expanded to reach solution:", num_expanded)
            yield context
        else:
            rem_cons = [c for c in constraints if c not in to_eval] # constraints involving unset variables
            var = var_order[0]
            rem_vars = var_order[1:]
            for val in var.domain:
                if display:
                    print("Setting", var, "to", val)
                num_expanded += 1
                rem_context = context|{var:val}
                # apply forward checking on remaining variables
                if len(var_order) > 1:
                    rem_vars_original = list((v, list(v.domain.copy())) for v in rem_vars)
                    if display:
                        print("Original domains:", rem_vars_original)
                    # constraints that can't already be evaluated in rem_cons
                    rem_cons_ff = [c for c in constraints if c in rem_cons and not can_evaluate(c, rem_context)]
                    for rem_var in rem_vars:
                        # constraints that can be evaluated by adding a value of rem_var to rem_context
                        rem_to_eval = {c for c in rem_cons_ff if can_evaluate(c, rem_context|{rem_var: rem_var.domain[0]})}
                        # new domain for rem_var are the values for which all newly evaluable constraints hold
                        rem_vals = rem_var.domain.copy()
                        for rem_val in rem_var.domain:
                            # no constraint with rem_var in the existing context can be violated
                            for c in rem_to_eval:
                                if not c.holds(rem_context|{rem_var: rem_val}):
                                    if rem_val in rem_vals:
                                        rem_vals.remove(rem_val)
                        rem_var.domain = rem_vals
                        # order remaining variables by MRV
                        rem_vars.sort(key=lambda v: len(v.domain))
                    if display:
                        print("After forward checking:", list((v, v.domain) for v in rem_vars))
                if rem_vars == [] or all(len(rem_var.domain) > 0 for rem_var in rem_vars):
                    yield from mrv_dfs_solver(rem_cons, context|{var:val}, rem_vars)
                # restore original domains if changed through forward checking
                if len(var_order) > 1:
                    if display:
                        print("Restoring original domain", rem_vars_original)
                    for (v, domain) in rem_vars_original:
                        v.domain = domain
            if display:
                print("Nodes expanded so far:", num_expanded)

def mrv_dfs_solve_all(csp, var_order=None):
    """ depth-first CSP solver to return a list of all solutions to csp """
    global num_expanded
    num_expanded = 0
    if var_order == None:    # order variables by MRV
        var_order = list(csp.variables)
        var_order.sort(key=lambda var: len(var.domain))
    return list(mrv_dfs_solver(csp.constraints, {}, var_order))

def mrv_dfs_solve1(csp, var_order=None):
    """ depth-first CSP solver """
    global num_expanded
    num_expanded = 0
    if var_order == None:    # order variables by MRV
        var_order = list(csp.variables)
        var_order.sort(key=lambda var: len(var.domain))
    for sol in mrv_dfs_solver(csp.constraints, {}, var_order):
        return sol  #return first one

Run this on the N-Queens problem.

In [22]:
from cspExamples import n_queens

# call the Depth-First Search solver
print(mrv_dfs_solve1(n_queens(4)))
# print(mrv_dfs_solve_all(n_queens(4)))

Nodes expanded to reach solution: 8
{R0: 1, R1: 3, R2: 2, R3: 0}


Run some experiments to compare the Depth-First constraint solver with and without forward checking and/or minimum remaining value heuristic.

### 4. Domain Splitting with Arc Consistency

The more complex constraint solver AIPython is based on domain splitting and checking for arc consistency at each step.

To implement this, the CSP is cast as a search problem where the actions are splitting the domain of one variable into two parts.

In [23]:
from searchGeneric import Searcher
from cspConsistency import Search_with_AC_from_CSP, select
from cspExamples import n_queens

sol = Searcher(Search_with_AC_from_CSP(n_queens(4))).search()
if sol:
    print("Solution:", {v: select(d) for (v,d) in sol.end().items()})

Solution: {R0: {0, 1, 2, 3}, R1: {0, 1, 2, 3}, R2: {0, 1, 2, 3}, R3: {0, 1, 2, 3}} --> {R0: {0, 1}, R1: {2, 3}, R2: {0, 1, 2}, R3: {0, 1, 2, 3}} --> {R0: {1}, R1: {3}, R2: {2}, R3: {0}} (cost: 2)
 3 paths have been expanded and 1 paths remain in the frontier
Solution: {R0: 1, R1: 3, R2: 2, R3: 0}


Examine the code in `cspConsistency.py` to see how the variable for splitting is chosen.

### 5. Stochastic Local Search

The final constraint solving method is Stochastic Local Search.

This is not guaranteed to find a solution within a given number of steps.

Run the code repeatedly and observe the differences in the number of steps to find a solution.

Compare the behaviour to the exact constraint solvers.

In [24]:
from cspSLS import SLSearcher
from cspExamples import n_queens

SLSearcher(n_queens(4)).search(1000, 1.0) # Fully greedy

Solution found: {R0: 2, R1: 1, R2: 3, R3: 0} in 11 steps


11

The parameters define the balance between greedily choosing a step and choosing variables based on the number of constraint violations.

What do you expect to happen? 

Run the algorithm with different values of the parameters and observe the differences.

Was your hypothesis correct?

In [25]:
from cspSLS import SLSearcher
from cspExamples import n_queens

SLSearcher(n_queens(4)).search(1000, 0.7) # Mostly greedy
SLSearcher(n_queens(4)).search(1000, 0) # Choose variable based only on conflicts

Solution found: {R0: 3, R1: 1, R2: 0, R3: 2} in 4 steps
Solution found: {R0: 3, R1: 1, R2: 0, R3: 2} in 97 steps


97